In [1]:
import os
import pandas as pd
import json
import openai
from openai import OpenAI
from typing import List
import re

In [2]:
client = 

In [3]:
goal_statement = "Goal - Develop an innovative extrinsic self healing coating with a one trigger component healing agent for corrosion protection of wind offshore structures. Self healing coating should be scalable for large scale structures"

constraint_list = """ 1) The material should incorporate a self-healing mechanism triggered by a simple environmental factor (e.g., water) to facilitate immediate and efficient damage recovery in corrosion-prone environments.
 2) The self-healing material should allow multiple healing events, utilizing a vascular-like structure to ensure a continuous supply of the healing agent to damaged areas.
 3) The coating must maintain its structural integrity and protective capabilities even after mechanical damage, preventing corrosion from initiating at the damaged site.
 4) The material should be compatible with scalable application techniques, such as spray-painting, to enable large-scale implementation in industrial environments.
 5) The healing mechanism should not rely on complex multi-component reactions but rather a single-component system, reducing complexity in formulation and ensuring a uniformhealing response."""

# prompt = f"""{goal_statement} \n\n Constraints - \n{constraint_list}.\n
# Provide me 20 innovative suggestions that will help achieve the above goal and satisfy all of the above mentioned constraints. 
# Provide reason for each suggestion. The suggestions must be in the below mentioned format in a JSON object. For example:\n
# {{Suggestion_1: 
#     Materials: 
#     Methods to develop the materials suggested: 
#     Reasoning:
#     ,
# Suggestion_20: 
#     Materials: 
#     Methods to develop the materials suggested: 
#     Reasoning:}}"""
    
# critic_prompt = f"""{goal_statement}\n\n {constraint_list} \n\n SUggestions:\n{chat_history[-1]}. \n\n Given the above goal statement, constraints and suggestions about materials design and discovery, evaluate each response and generate detailed feedback which will help the suggestion generation process to generate suggestions such that they help achieve goal statement and satisfy all the constraints strictly. The detailed feedback should be in the below JSON format strictly:
# {{"Feedback for Suggestion_1":
# meets the goal statement and satisfies all constraints strictly: "YES/NO"
# Reasoning:" ",
# "Feedback for Suggestion_20":
# meets the goal statement and satisfies all constraints strictly: "YES/NO"
# Reasoning:" ",
# "Overall Feedback for improvement": " " ]]
# }}
# """
# print(f'critic_prompt-----> {critic_prompt}') 


#################################################################################Functions For Prompt Construction#################################################################

In [4]:
def construct_prompt_for_hypotheses_generator(goal_statement:str, constraint_list:str):
    prompt = f"""{goal_statement} \n\n Constraints:- \n{constraint_list}.\n
Provide me 20 innovative suggestions that will help achieve the above goal while satisfying all of the above mentioned constraints strictly. 
Provide reason for each suggestion. The suggestions must be in the below mentioned format in a JSON object. For example:\n
{{Suggestion_1: 
    Materials: 
    Methods_to_develop_the_materials_suggested: 
    Reasoning:
    ,
Suggestion_20: 
    Materials: 
    Methods_to_develop_the_materials_suggested: 
    Reasoning: }}"""
    return prompt

def construct_critic_prompt(goal_statement:str, constraint_list:str, chat_history:str):
    critic_prompt = f"""{goal_statement}\n\nConstraints:-\n{constraint_list}\n\nSuggestions:\n{chat_history}Given the above goal statement, constraints and suggestions about materials design and discovery, evaluate each suggestion and generate detailed feedback which will help the suggestion generation process to generate suggestions such that they help achieve goal statement and satisfy all the constraints strictly. The detailed feedback should be in the below JSON format strictly:
    {{"Feedback_for_suggestion_1":
    Meets_the_goal_statement_and_satisfies_all_constraints_strictly: "YES/NO"
    Reasoning:" ",
    "Feedback_for_suggestion_20":
    Meets_the_goal_statement_and_satisfies_all_constraints_strictly: "YES/NO"
    Reasoning:" ",
    "Overall_Feedback_for_improvement": " " ]]
    }}
    """
    return critic_prompt

def construct_feedback_prompt(feedback):
    feedback_prompt = f"""Below provided is the feedback you gave for each of the initial suggestions generated and an overall feedback for the improvement of future suggestion generations\n{feedback}.Refine your suggestions based on the feedback accordingly to meet the goal statement and satisfy all the constraints strictly. The suggestions must be in the below mentioned format in a JSON object. For example:\n
{{Suggestion_1: 
    Materials: 
    Methods_to_develop_the_materials_suggested: 
    Reasoning:
    ,
Suggestion_20: 
    Materials: 
    Methods_to_develop_the_materials_suggested: 
    Reasoning:}}"""  
    return feedback_prompt

def construct_feedback_prompt_for_refined_hypotheses(feedback_history,chat_history):
    feedback_prompt = f"""Below provided is the feedback you gave for the initial suggestions\n{feedback_history}. Below are the refined suggestions based on the feedback\n{chat_history}. Now evaluate each refined suggestion and provide detailed feedback which will help the suggestion generation process to generate suggestions such that they help achieve goal statement and satisfy all the constraints strictly. The detailed feedback should be in the below JSON format strictly:
    {{"Feedback_for_suggestion_1":
    Meets_the_goal_statement_and_satisfies_all_constraints_strictly: "YES/NO"
    Reasoning:" ",
    "Feedback_for_suggestion_20":
    Meets_the_goal_statement_and_satisfies_all_constraints_strictly: "YES/NO"
    Reasoning:" ",
    "Overall_Feedback_for_improvement": " " ]]
    }}
    """
    return feedback_prompt

In [5]:
print(f'prompt----->\n{construct_prompt_for_hypotheses_generator(goal_statement, constraint_list)}')

prompt----->
Goal - Develop an innovative extrinsic self healing coating with a one trigger component healing agent for corrosion protection of wind offshore structures. Self healing coating should be scalable for large scale structures 

 Constraints:- 
 1) The material should incorporate a self-healing mechanism triggered by a simple environmental factor (e.g., water) to facilitate immediate and efficient damage recovery in corrosion-prone environments.
 2) The self-healing material should allow multiple healing events, utilizing a vascular-like structure to ensure a continuous supply of the healing agent to damaged areas.
 3) The coating must maintain its structural integrity and protective capabilities even after mechanical damage, preventing corrosion from initiating at the damaged site.
 4) The material should be compatible with scalable application techniques, such as spray-painting, to enable large-scale implementation in industrial environments.
 5) The healing mechanism shoul

#################################################################################Functions For Agent Construction#################################################################

In [5]:
def expert_list_generator(goal_statement):
    completion = client.chat.completions.create(
        model = 'gpt-4o',
        temperature = 0.7,
        messages = [
            {
                'role': 'system',
                'content': f'You are an helpful assistant'
            },
            {
                'role': 'user',
                'content': f'Generate a list of experts required to achieve the below mentioned goal:\n{goal_statement}. Just list the top 5 experts in the format "Expert_1, Expert_2, Expert_3, Expert_4, Expert_5"'
                }
              ]
    )
    return completion.choices[0].message.content

In [7]:
expert_list = expert_list_generator(goal_statement)
print(f'expert_list----->\n{expert_list}')

expert_list----->
Material Scientist, Chemical Engineer, Corrosion Specialist, Coatings Formulation Expert, Scale-Up Process Engineer


In [6]:
def hypothesis_generator(expert_list,prompt,feedback=None,chat_history=None):
    if feedback == None and chat_history == None:
        completion = client.chat.completions.create(
            model = 'gpt-4o',
            temperature = 0.7,
            messages = [
                {
                    'role': 'system',
                    'content': f'You are an innovative {expert_list} capable of doing impactful materials discovery and design'
                },
                {
                    'role': 'user',
                    'content': prompt
                },
            ],
            response_format = {"type": "json_object"}
        )
    else:
        completion = client.chat.completions.create(
            model = 'gpt-4o',
            temperature = 0.7,
            messages = [
                {
                    'role': 'system',
                    'content': f'You are an innovative {expert_list} capable of doing impactful materials discovery and design'
                },
                {
                    'role': 'user',
                    'content': prompt
                },
                {
                    'role': 'assistant',
                    'content': chat_history
                },
                {
                    'role': 'user',
                    'content': feedback
                }
            ],
            response_format = {"type": "json_object"}
        )
    return completion.choices[0].message.content

In [9]:
hypothese = hypothesis_generator(expert_list,construct_prompt_for_hypotheses_generator(goal_statement, constraint_list))
print(f'hypothese----->\n{hypothese}')

hypothese----->
{
"Suggestion_1": {
    "Materials": "Polyurethane matrix embedded with microcapsules containing hydrophobic healing agents",
    "Methods_to_develop_the_materials_suggested": "Synthesize microcapsules containing healing agents, disperse them into a polyurethane matrix, and apply the coating via spray-painting.",
    "Reasoning": "Polyurethane provides excellent mechanical properties and environmental resistance. The microcapsules will release healing agents upon encountering water, initiating self-healing."
},
"Suggestion_2": {
    "Materials": "Epoxy resin with encapsulated liquid metal healing agents",
    "Methods_to_develop_the_materials_suggested": "Encapsulate a liquid metal (e.g., gallium-indium alloy) in microcapsules, mix with epoxy resin, and apply the mixture using spray-painting techniques.",
    "Reasoning": "Liquid metals can fill cracks and bind strongly to the epoxy matrix, ensuring structural integrity and multiple self-healing cycles upon exposure to 

In [10]:
print(type(hypothese))

<class 'str'>


In [7]:
def critic_1(expert_list,critic_prompt,feedback_history,refined_feedback_prompt):
    if feedback_history==None and refined_feedback_prompt==None:
        completion = client.chat.completions.create(
            model = 'gpt-4o',
            temperature = 0.7,
            messages = [
                {
                    'role': 'system',
                    'content': f'You are an expert {expert_list} capable of doing impactful materials discovery and design. Given a goal statement, additional constraints, and a list of suggestions about materials design and discovery, your task is to evaluate each suggestion such that it meets the goal statement and satisfies all the constraints strictly. '
                },
                {
                    'role': 'user',
                    'content': critic_prompt
                }
            ],
            response_format = {"type": "json_object"}
        )
    else: 
        completion = client.chat.completions.create(
        model = 'gpt-4o',
        temperature = 0.7,
        messages = [
            {
                'role': 'system',
                'content': f'You are an expert {expert_list} capable of doing impactful materials discovery and design. Given a goal statement, additional constraints, and a list of suggestions about materials design and discovery, your task is to evaluate each suggestion such that it meets the goal statement and satisfies all the constraints strictly.'
            },
            {
                'role': 'user',
                'content': critic_prompt
            },
            {
                'role': 'assistant',
                'content': feedback_history
            },
            {
                'role': 'user',
                'content': refined_feedback_prompt
            }
        ],
        response_format = {"type": "json_object"}
    )
    return completion.choices[0].message.content


##############################################################################Functions to process text####################################################################################

In [8]:
def json_to_text(json_obj):
    output_text = ""
    for key, value in json_obj.items():
        suggestion_details = (
            f"{key.replace('_', ' ')}:\n"
            f"Materials:{''.join(value['Materials'])}\n"
            f"Methods_to_develop_the_materials_suggested:{''.join(value['Methods_to_develop_the_materials_suggested'])}\n"
            f"Reasoning:{value[f'Reasoning']}\n\n"
        )
        output_text += suggestion_details
    return output_text

In [13]:
output_text = json_to_text(json.loads(hypothese))
print(output_text)

Suggestion 1:
Materials:Polyurethane matrix embedded with microcapsules containing hydrophobic healing agents
Methods_to_develop_the_materials_suggested:Synthesize microcapsules containing healing agents, disperse them into a polyurethane matrix, and apply the coating via spray-painting.
Reasoning:Polyurethane provides excellent mechanical properties and environmental resistance. The microcapsules will release healing agents upon encountering water, initiating self-healing.

Suggestion 2:
Materials:Epoxy resin with encapsulated liquid metal healing agents
Methods_to_develop_the_materials_suggested:Encapsulate a liquid metal (e.g., gallium-indium alloy) in microcapsules, mix with epoxy resin, and apply the mixture using spray-painting techniques.
Reasoning:Liquid metals can fill cracks and bind strongly to the epoxy matrix, ensuring structural integrity and multiple self-healing cycles upon exposure to water.

Suggestion 3:
Materials:Acrylic polymer with embedded hollow fibers containin

In [14]:
critic_prompt_ = construct_critic_prompt(goal_statement, constraint_list, output_text)
print(f'critic_prompt----->\n{critic_prompt_}')

critic_prompt----->
Goal - Develop an innovative extrinsic self healing coating with a one trigger component healing agent for corrosion protection of wind offshore structures. Self healing coating should be scalable for large scale structures

Constraints:-
 1) The material should incorporate a self-healing mechanism triggered by a simple environmental factor (e.g., water) to facilitate immediate and efficient damage recovery in corrosion-prone environments.
 2) The self-healing material should allow multiple healing events, utilizing a vascular-like structure to ensure a continuous supply of the healing agent to damaged areas.
 3) The coating must maintain its structural integrity and protective capabilities even after mechanical damage, preventing corrosion from initiating at the damaged site.
 4) The material should be compatible with scalable application techniques, such as spray-painting, to enable large-scale implementation in industrial environments.
 5) The healing mechanism s

In [15]:
feedback = critic_1(expert_list,critic_prompt_,None,None)
print(f'feedback----->\n{feedback}')


feedback----->
{
    "Feedback_for_suggestion_1": {
        "Meets_the_goal_statement_and_satisfies_all_constraints_strictly": "NO",
        "Reasoning": "While polyurethane provides good mechanical properties and environmental resistance, the use of hydrophobic healing agents may not be ideal for multiple healing cycles. Additionally, hydrophobic agents may not be efficiently released upon water exposure."
    },
    "Feedback_for_suggestion_2": {
        "Meets_the_goal_statement_and_satisfies_all_constraints_strictly": "NO",
        "Reasoning": "Liquid metals like gallium-indium alloys may provide good self-healing properties, but their encapsulation and consistent release upon water exposure could be complex. Additionally, liquid metals may pose scalability issues for large-scale spray-painting applications."
    },
    "Feedback_for_suggestion_3": {
        "Meets_the_goal_statement_and_satisfies_all_constraints_strictly": "YES",
        "Reasoning": "Acrylic polymer with embedde

In [9]:
# def process_feedback(feedback_dict):
#     feedback_for_negative_suggestions = ""
#     final_answer = "Yes"
#     for key, value in feedback_dict.items():
#         if type(value)==dict:
#             for k,v in value.items():
#                 if k == 'Meets_the_goal_statement_and_satisfies_all_constraints_strictly':
#                     if v == 'NO':
#                         feedback_for_negative_suggestions += f"Suggestion_{key.split('_')[-1]} does not meet the goal statement and satisfies all constraints strictly beacuse {value['Reasoning']}\n"
#                         final_answer = "NO"
#         else:
#             overall_feedback = value     
#     return feedback_for_negative_suggestions, overall_feedback, final_answer

def process_feedback_extract_final_answer(feedback):
    suggestions_with_no = 0
    processed_feedback = ""
    final_answer = "Yes"
    for key, value in feedback.items():
        if key.startswith("Feedback_for_suggestion"):
            suggestion_num = key.split('_')[-1]
            Meets_the_goal_statement_and_satisfies_all_constraints_strictly = value.get('Meets_the_goal_statement_and_satisfies_all_constraints_strictly', 'N/A')
            if Meets_the_goal_statement_and_satisfies_all_constraints_strictly == 'NO':
                suggestions_with_no += 1
                final_answer = "NO"
            Reasoning = value.get('Reasoning', 'N/A')
            processed_feedback += f"Feedback_for_suggestion_{suggestion_num}:\nMeets_the_goal_statement_and_satisfies_all_constraints_strictly:{Meets_the_goal_statement_and_satisfies_all_constraints_strictly}.\nReasoning: {Reasoning}\n\n"
        elif key.startswith("Overall_Feedback_for_improvement"):
            processed_feedback += f"Overall Feedback_for_future_suggestion_improvement: {value}\n"
    return processed_feedback, final_answer, suggestions_with_no
    
        

In [24]:
feedback_dict = json.loads(feedback)
feedback_dict
processed_feedback, final_answer = process_feedback_extract_final_answer(feedback_dict)

print(f'processed_feedback----->\n{processed_feedback}')
print(f'final_answer----->\n{final_answer}')

processed_feedback----->
Feedback_for_suggestion_1:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:NO.
Reasoning: While polyurethane provides good mechanical properties and environmental resistance, the use of hydrophobic healing agents may not be ideal for multiple healing cycles. Additionally, hydrophobic agents may not be efficiently released upon water exposure.

Feedback_for_suggestion_2:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:NO.
Reasoning: Liquid metals like gallium-indium alloys may provide good self-healing properties, but their encapsulation and consistent release upon water exposure could be complex. Additionally, liquid metals may pose scalability issues for large-scale spray-painting applications.

Feedback_for_suggestion_3:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Reasoning: Acrylic polymer with embedded hollow fibers containing a hydrophilic healing agent meets the constraints. Acrylic polymers are dur

In [10]:
chat_history = []
feedback_history = []
initial_feedback = None
final_answer = ""
feedback_prompt = None
refined_feedback_prompt = None
text_from_feedback = None
suggestions_no_list = []
expert_list = expert_list_generator(goal_statement)

prompt = construct_prompt_for_hypotheses_generator(goal_statement, constraint_list)
print(f'prompt----->')
generated_hypotheses = hypothesis_generator(expert_list,prompt,feedback_prompt)
generated_hypotheses = json.loads(generated_hypotheses)
# generated_hypotheses = json_to_text(generated_hypotheses)
print(f'generated_hypotheses----->')

if len(generated_hypotheses.keys())==20:
    generated_hypotheses = json_to_text(generated_hypotheses)
    chat_history.append(generated_hypotheses)
    critic_prompt = construct_critic_prompt(goal_statement, constraint_list, chat_history[-1])
    feedback_from_critic_1 = critic_1(expert_list,critic_prompt,initial_feedback,refined_feedback_prompt)
    feedback_from_critic_1 = json.loads(feedback_from_critic_1)
    feedback_from_critic_1,final_answer,suggestions_with_no = process_feedback_extract_final_answer(feedback_from_critic_1)
    feedback_history.append(feedback_from_critic_1)
    print('suggestions_with_no----->',suggestions_with_no)
    print(f'final_answer-----> {final_answer}')
    attempts = 0       
    while final_answer!="Yes" and attempts<5:      
        feedback_prompt = construct_feedback_prompt(feedback_from_critic_1)
        print('==============>Constructing feedback prompt for refined hypotheses')
        refined_hypotheses = hypothesis_generator(expert_list, prompt, feedback_prompt, chat_history[-1])
        refined_hypotheses = json.loads(refined_hypotheses)
        print('===================>refined hypothesis generated')
        chat_history.append(json_to_text(refined_hypotheses))
        feedback_prompt_for_refined_hypothesis = construct_feedback_prompt_for_refined_hypotheses(feedback_history[-1], chat_history[-1])
        print('===================>Constructing feedback prompt for refined hypotheses')
        refined_feedback = critic_1(expert_list,critic_prompt,feedback_history[-1],feedback_prompt_for_refined_hypothesis)
        refined_feedback = json.loads(refined_feedback)
        print('===================>feedback received for refined hypotheses')
        feedback_from_critic_1,final_answer,suggestions_with_no = process_feedback_extract_final_answer(refined_feedback)
        feedback_history.append(feedback_from_critic_1)
        suggestions_no_list.append(suggestions_with_no)
        print(f'suggestions_with_no-----> {suggestions_with_no}')
        attempts += 1 
        print(f'attempts-----> {attempts}')
    if final_answer=="Yes":
        print("Suggestions are generated properly")
else:
    print("Suggestions are not generated properly. Please try again")

average_suggestions_with_no = sum(suggestions_no_list) / len(suggestions_no_list)



prompt----->
generated_hypotheses----->
suggestions_with_no-----> 13
final_answer-----> NO
==============>Constructing feedback prompt for refined hypotheses
===================>refined hypothesis generated
===================>Constructing feedback prompt for refined hypotheses
===================>feedback received for refined hypotheses
attempts-----> 1
==============>Constructing feedback prompt for refined hypotheses
===================>refined hypothesis generated
===================>Constructing feedback prompt for refined hypotheses
===================>feedback received for refined hypotheses
attempts-----> 2
Suggestions are generated properly


ZeroDivisionError: division by zero

In [11]:

print(feedback_history[-1])


Feedback_for_suggestion_1:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Reasoning: Microencapsulated polyurethane prepolymers can polymerize in the presence of water, providing a single-component healing mechanism that is simple and effective. The use of microcapsules can support multiple healing events and the system is compatible with scalable techniques like spray-painting.

Feedback_for_suggestion_2:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Reasoning: Water-reactive siloxane-based microcapsules can form a durable protective layer upon water exposure, ensuring a reliable self-healing mechanism. These systems support multiple healing events and can be applied using scalable methods such as spray-painting.

Feedback_for_suggestion_3:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Reasoning: Hydrogel encapsulated healing agents swell and release their contents upon water contact, providing a simple and efficient 

In [ ]:
chat_history =
feedback_history =

In [11]:
print(feedback_history[0])

Feedback_for_suggestion_1:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Reasoning: Polyurethane provides good mechanical properties and environmental resistance, while the hydrogel microcapsules can rupture upon exposure to water, releasing the healing agent. This satisfies the requirement for a simple environmental trigger and allows for multiple healing events with a vascular-like structure. The material is compatible with scalable application techniques like spray-painting.

Feedback_for_suggestion_2:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:NO.
Reasoning: While epoxy resin provides high adhesion and durability, the use of sodium polyacrylate microcapsules containing a hydrophilic monomer introduces complexity in the healing mechanism. The swelling and polymerization process can be considered a multi-component reaction, which goes against the single-component system requirement.

Feedback_for_suggestion_3:
Meets_the_goal_statement_and_s

In [12]:
print(feedback_history[1])

Feedback_for_suggestion_1:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Reasoning: Polyurethane provides good mechanical properties and environmental resistance. The hydrogel microcapsules can rupture upon exposure to water, releasing the healing agent. This satisfies the requirement for a simple environmental trigger and allows for multiple healing events with a vascular-like structure. The material is compatible with scalable application techniques like spray-painting.

Feedback_for_suggestion_2:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Reasoning: Acrylic resin is durable and easily applicable via spray-painting. The microtubules containing a water-reactive curing agent meet the requirements for a simple environmental trigger and allow for multiple healing events. This system maintains structural integrity and protective capabilities.

Feedback_for_suggestion_3:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Re

In [11]:
print(feedback_history[-1])

{'Feedback_for_suggestion_1': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'YES', 'Reasoning': 'Hydrogel-filled microtubes can provide multiple healing events and are water-triggered. Embedding these microtubes into a polymer coating can maintain structural integrity and allow for scalable application techniques, satisfying all the constraints.'}, 'Feedback_for_suggestion_2': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'YES', 'Reasoning': 'Microencapsulated self-healing agents with vascular networks ensure a continuous supply of the healing agent and multiple healing events. The use of 3D printing or electrospinning for vascular structures can maintain structural integrity and allow for scalable application techniques.'}, 'Feedback_for_suggestion_3': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'YES', 'Reasoning': 'Water-reactive polyurethane integrated with microvascular networks can form a protective barrier upon w

In [11]:
print(feedback_history[-1])

{'Feedback_for_suggestion_1': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'YES', 'Reasoning': 'Microencapsulated polyurethane prepolymers can polymerize upon water contact, providing a simple trigger for the healing mechanism. The microcapsules can support multiple healing events if properly distributed and formulated. The material is compatible with scalable application techniques like spray-painting.'}, 'Feedback_for_suggestion_2': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'YES', 'Reasoning': 'Polysulfide-based coatings can polymerize in the presence of water, providing a straightforward healing trigger. Polysulfides form strong, flexible bonds, supporting multiple healing events and maintaining structural integrity. They can be applied using scalable techniques like spray-painting.'}, 'Feedback_for_suggestion_3': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'YES', 'Reasoning': 'Water-based polyurethane dispersi

In [30]:
processed_feedback,final_answer = process_feedback_extract_final_answer(feedback_history[-1])
print(processed_feedback)
print(final_answer)


Feedback_for_suggestion_1:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Reasoning: The polyurethane matrix with microencapsulated sodium silicate meets the goal of an extrinsic self-healing coating. The water-reactive sodium silicate ensures a simple, single-component healing mechanism. The microencapsulation allows for multiple healing events, and polyurethane is suitable for scalable application techniques like spray-painting.

Feedback_for_suggestion_2:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:NO.
Reasoning: While epoxy resin with embedded hydrophilic fibers provides strong adhesion and corrosion resistance, the method relies on fiber swelling rather than a single-component healing agent. This could lead to inconsistencies in the healing response and may not support multiple healing events as effectively.

Feedback_for_suggestion_3:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:NO.
Reasoning: PVDF with hydrogel micropa

In [23]:
print(processed_feedback)

Feedback_for_suggestion_1:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:YES.
Reasoning: The polyurethane matrix with microencapsulated sodium silicate meets the goal of an extrinsic self-healing coating. The water-reactive sodium silicate ensures a simple, single-component healing mechanism. The microencapsulation allows for multiple healing events, and polyurethane is suitable for scalable application techniques like spray-painting.
Feedback_for_suggestion_2:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:NO.
Reasoning: While epoxy resin with embedded hydrophilic fibers provides strong adhesion and corrosion resistance, the method relies on fiber swelling rather than a single-component healing agent. This could lead to inconsistencies in the healing response and may not support multiple healing events as effectively.
Feedback_for_suggestion_3:
Meets_the_goal_statement_and_satisfies_all_constraints_strictly:NO.
Reasoning: PVDF with hydrogel micropart

In [17]:
print(feedback_history[0])

{'Feedback_for_suggestion_1': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'YES', 'Reasoning': 'The polyurethane matrix with microencapsulated sodium silicate meets the goal of an extrinsic self-healing coating. The water-reactive sodium silicate ensures a simple, single-component healing mechanism. The microencapsulation allows for multiple healing events, and polyurethane is suitable for scalable application techniques like spray-painting.'}, 'Feedback_for_suggestion_2': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'NO', 'Reasoning': 'While epoxy resin with embedded hydrophilic fibers provides strong adhesion and corrosion resistance, the method relies on fiber swelling rather than a single-component healing agent. This could lead to inconsistencies in the healing response and may not support multiple healing events as effectively.'}, 'Feedback_for_suggestion_3': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'NO', 'R

In [15]:
refined_hypotheses_ = hypothesis_generator(expert_list, prompt, feedback_history[-1], chat_history[-1])
refined_hypotheses_

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages[3].content[0]': expected an object, but got a string instead.", 'type': 'invalid_request_error', 'param': 'messages[3].content[0]', 'code': 'invalid_type'}}

In [13]:
print(feedback_prompt_for_refined_hypothesis)

Below provided is the feedback you gave for the initial suggestions
{'Feedback_for_suggestion_1': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'YES', 'Reasoning': 'The polyurethane matrix with microencapsulated sodium silicate meets the goal of an extrinsic self-healing coating. The water-reactive sodium silicate ensures a simple, single-component healing mechanism. The microencapsulation allows for multiple healing events, and polyurethane is suitable for scalable application techniques like spray-painting.'}, 'Feedback_for_suggestion_2': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'NO', 'Reasoning': 'While epoxy resin with embedded hydrophilic fibers provides strong adhesion and corrosion resistance, the method relies on fiber swelling rather than a single-component healing agent. This could lead to inconsistencies in the healing response and may not support multiple healing events as effectively.'}, 'Feedback_for_suggestion_3': {'Meets_

In [36]:
chat_history

['Suggestion 1:\nMaterials:Hydrophilic Polyurethane with Encapsulated Epoxy Resin\nMethods_to_develop_the_materials_suggested:Synthesize a hydrophilic polyurethane matrix and embed microcapsules containing epoxy resin. Utilize interfacial polymerization to create the microcapsules.\nReasoning:Hydrophilic polyurethane will react with water, causing the capsules to rupture and release the epoxy resin, which will then polymerize to heal the damage.\n\nSuggestion 2:\nMaterials:Polyvinyl Alcohol (PVA) with Embedded Microcapsules of UV-Curable Resin\nMethods_to_develop_the_materials_suggested:Create a PVA coating and incorporate microcapsules containing UV-curable resin. Use emulsion polymerization to produce the microcapsules.\nReasoning:PVA swells in water, causing the capsules to rupture and release the UV-curable resin, which can then be cured by ambient UV light.\n\nSuggestion 3:\nMaterials:Hydrogel-Infused Epoxy Coating with Encapsulated Polyurethane Prepolymer\nMethods_to_develop_the_

In [21]:
print(feedback_prompt_for_refined_hypothesis)

Below provided is the feedback you gave for the initial suggestions
{'Feedback_for_suggestion_1': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'NO', 'Reasoning': 'While hydrophilic polyurethane and encapsulated epoxy resin can provide self-healing properties triggered by water, epoxy resin generally involves multi-component curing reactions. This does not strictly satisfy the single-component system constraint.'}, 'Feedback_for_suggestion_2': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'NO', 'Reasoning': 'UV-curable resin requires UV light as an additional trigger, which is not always available in offshore environments, making it less reliable. Additionally, the healing event involves a multi-component reaction, which does not meet the single-component system requirement.'}, 'Feedback_for_suggestion_3': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'NO', 'Reasoning': 'The hydrogel and polyurethane prepolymer combinati

In [12]:
print(chat_history[0])

Suggestion 1:
Materials:Polyurethane matrix with microencapsulated hydrophilic healing agent
Methods_to_develop_the_materials_suggested:Encapsulate the hydrophilic agent within microcapsules and disperse these microcapsules uniformly in the polyurethane matrix.
Reasoning:Polyurethane provides excellent mechanical properties and can be easily spray-painted. The hydrophilic agent will be activated by water ingress, facilitating the self-healing process.

Suggestion 2:
Materials:Epoxy resin with embedded water-activated polymer spheres
Methods_to_develop_the_materials_suggested:Synthesize water-activated polymer spheres and embed them within an epoxy resin matrix.
Reasoning:Epoxy resins are highly durable and offer good adhesion. The water-activated polymer spheres will swell and heal the damage upon water exposure.

Suggestion 3:
Materials:Acrylic polymer with vascular channels filled with a water-sensitive healing agent
Methods_to_develop_the_materials_suggested:Incorporate vascular cha

In [29]:
print(feedback_history[-1])

{'Feedback_for_suggestion_1': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'NO', 'Reasoning': 'While hydrophilic polyurethane and encapsulated epoxy resin can provide self-healing properties triggered by water, epoxy resin generally involves multi-component curing reactions. This does not strictly satisfy the single-component system constraint.'}, 'Feedback_for_suggestion_2': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'NO', 'Reasoning': 'UV-curable resin requires UV light as an additional trigger, which is not always available in offshore environments, making it less reliable. Additionally, the healing event involves a multi-component reaction, which does not meet the single-component system requirement.'}, 'Feedback_for_suggestion_3': {'Meets_the_goal_statement_and_satisfies_all_constraints_strictly': 'NO', 'Reasoning': 'The hydrogel and polyurethane prepolymer combination can heal upon water exposure. However, polyurethane prepolymer ma

In [30]:
type(feedback_history[-1])

dict

In [25]:
str(feedback_history[-1])

'{\'Feedback_for_suggestion_1\': {\'Meets_the_goal_statement_and_satisfies_all_constraints_strictly\': \'NO\', \'Reasoning\': \'While hydrophilic polyurethane and encapsulated epoxy resin can provide self-healing properties triggered by water, epoxy resin generally involves multi-component curing reactions. This does not strictly satisfy the single-component system constraint.\'}, \'Feedback_for_suggestion_2\': {\'Meets_the_goal_statement_and_satisfies_all_constraints_strictly\': \'NO\', \'Reasoning\': \'UV-curable resin requires UV light as an additional trigger, which is not always available in offshore environments, making it less reliable. Additionally, the healing event involves a multi-component reaction, which does not meet the single-component system requirement.\'}, \'Feedback_for_suggestion_3\': {\'Meets_the_goal_statement_and_satisfies_all_constraints_strictly\': \'NO\', \'Reasoning\': \'The hydrogel and polyurethane prepolymer combination can heal upon water exposure. Howev

In [42]:
feedback_from_critic_1 = critic_1(expert_list,critic_prompt,feedback_history[-1],refined_feedback_prompt)
print(f'feedback_from_critic_1----->\n{feedback_from_critic_1}')

UnboundLocalError: cannot access local variable 'completion' where it is not associated with a value